In [10]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.prompt import PromptTemplate

llm=ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True, # 챗 모델 사용을 위해
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI Talking to a human"),
        MessagesPlaceholder(variable_name="history"), #메세지가 얼마나 많고 누구에게로 부터 왔는지 모르지만 placeholder가 memory class로 대체
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"] #메모리를 로드

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke(
        {"question": question}
    )
    memory.save_context({"input": question}, {"output": result.content}) # 사람과 AI메세지인 input과 output을 메모리에 저장
    return print(result)


In [11]:
invoke_chain("Hi, I am Sako")

content='Hello Sako! How can I assist you today?'


In [12]:
invoke_chain("I work at a startup called sue")

content="That's great to hear! How can I help you with your work at Sue?"


In [13]:
invoke_chain("What is my name?")

content='Your name is Sako.'
